## Segmentation 학습에서 dfl backbone을 적용


#### 1. 라이브러리 호출

In [1]:
import sys
sys.path.append('../')

In [2]:
import os, glob, random, cv2
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
import segmentation_models_pytorch as smp
import segmentation.model.metric as module_metric
import dfl_model.dfl_cnn as DFL


from segmentation.data_loader.dataloader import get_dataloader
from utils.data import get_datasize
from utils.visual import *
from albumentations.pytorch import transforms
from segmentation.model.loss import *
from segmentation.train import *
from pathlib import Path


: 

: 

#### 2. 시드고정

In [16]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

#### 3. 하이퍼 파라미터 설정

In [17]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

lr = 1e-3
batch_size = 16
num_epoch = 200
damage = 'dent'

train_dir = f'./dataset/{damage}/train/'
val_dir = f'./dataset/{damage}/valid/'
test_dir = f'./dataset/{damage}/test/'

mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)
image_size = 512

In [18]:
transform_train = A.Compose([
    A.Resize(image_size, image_size),
    A.HorizontalFlip(),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.Rotate((-10, 10), p=0.5, border_mode=cv2.BORDER_REFLECT,),
    A.Normalize(mean, std),
    transforms.ToTensorV2(transpose_mask=True)
])

transform_val = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(mean, std),
    transforms.ToTensorV2(transpose_mask=True)
])

transform_test = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(mean, std),
    transforms.ToTensorV2(transpose_mask=True)
])

In [19]:
train_dataloader = get_dataloader(train_dir, transform_train, batch_size, True)
val_dataloader = get_dataloader(val_dir, transform_val, batch_size, False)
test_dataloader = get_dataloader(test_dir, transform_test, batch_size, False)

In [4]:
import dfl_model.dfl_cnn as DFL

In [2]:
model_dfl = DFL.DFL_VGG16()
model_dfl

c:\Users\SUN\miniconda3\envs\env2022\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\SUN\miniconda3\envs\env2022\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to C:\Users\SUN/.cache\torch\hub\checkpoints\vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [5]:
model = DFL.DFL_RESNET50()
model

c:\Users\SUN\miniconda3\envs\env2022\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\SUN\miniconda3\envs\env2022\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DFL_RESNET50(
  (conv1_conv4): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Co

In [ ]:
# # 불러온 DFL의 weight 확인
# for name, param in model_dfl.named_parameters():
#     print(name)
#     print(param)

In [ ]:
# # 저장된 weight 가져오기
# model_dfl.load_state_dict(torch.load('/home/aiffel/Desktop/SUN/DFL-CNN/DFL_Model.pth'))

In [ ]:
# # 저장된 weight 불러온 뒤 모델의 weight 확인하기(위에와 비교해보기)
# for name, param in model_dfl.named_parameters():
#     print(name)
#     print(param)

In [ ]:
# segmentation 학습할 모델 불러오기
# import segmentation_models_pytorch as smp

# model_unet = smp.Unet(encoder_name='vgg16_bn')
# # model_unet

In [ ]:
# segmentation 모델의 파라미터 확인하기
# for name, param in model_unet.named_parameters():
#     print(name) # param만 찍어서 확인해도 ok
#     print(param)

weight 바꿔주기

In [ ]:
# for i in range(0,33):
#     try:
#         model_dfl.conv1_conv4[i].weight
#         model_unet.encoder.features[i].weight
#     except :
#         print(f'{i} : ', model_dfl.conv1_conv4[i])
#         print(f'{i} : ', model_unet.encoder.features[i])
#     else :
#         print(f'{i}')
#         #print(model_dfl.conv1_conv4[i])#,model_dfl.conv1_conv4[i].weight)
#         #print(model_unet.encoder.features[i])#,model_unet.encoder.features[i].weight)
#         model_unet.encoder.features[i].weight = model_dfl.conv1_conv4[i].weight
#         print('========================================================================')

# print("완료!")

In [ ]:
# for i in range(1,model_dfl.conv5.__len__()):
#     try:
#         model_dfl.conv5[i].weight
#         model_unet.encoder.features[i+33].weight
#     except :
#         print(f'{i} : ', model_dfl.conv5[i])
#         print(f'{i} : ', model_unet.encoder.features[i+33])
#     else :
#         print(f'{i}')
#         #print(model_dfl.conv5[i])#,model_dfl.conv1_conv4[i].weight)
#         #print(model_unet.encoder.features[i+33])#,model_unet.encoder.features[i].weight)
#         model_unet.encoder.features[i+33].weight = model_dfl.conv5[i].weight 
#         print('===============================================================================')


# print("완료!")

In [ ]:
# segmentation 모델에 weight 바뀌었는지 확인하기
# for name, param in model_unet.named_parameters():
#     print(name)
#     print(param)

In [20]:
# 원래 segmentation pretrained 불러올때 쓰던 코드

# model = smp.Unet(encoder_name='efficientnet-b0', encoder_weights='imagenet', in_channels=3, classes=1, activation=None)


In [ ]:
model = model.to(device)

In [21]:
criterion = nn.BCEWithLogitsLoss().to(device)

optimizer = optim.SGD( model.parameters(), 
                        momentum=0.9, 
                        lr=lr )

scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts( optimizer=optimizer, 
                                                            T_0=20, 
                                                            T_mult=2, 
                                                            eta_min=1e-5 )

metrics = [getattr(module_metric, met) for met in ['IOUscore', 'PixelAccuracy']]

#### 4. wandb config 생성

In [22]:
train_config = {}
train_config['Batch size'] = batch_size
train_config['Learning Rate'] = lr
train_config['Epochs'] = num_epoch
train_config['Image size'] = image_size

train_config['Loss fn'] = criterion.__class__.__name__
train_config['Optimizer'] = optimizer.__class__.__name__
train_config['LR Scheduler'] = scheduler.__class__.__name__
train_config['Metric'] = {str(idx+1) : metric for idx, metric in enumerate([metrics[i].__name__ for i in range(len(metrics))])}

In [23]:
save_dir = f"./saved/U-Net/{'_'.join([model.__dict__['name'].split('-')[1:][0].capitalize(), model.__dict__['name'].split('-')[-1].capitalize()])}_{damage}/"
print(save_dir)

trainer = Trainer( model, criterion, metrics, optimizer, device, num_epoch, save_dir, mean, std,
                  data_loader=train_dataloader, valid_data_loader=val_dataloader, test_data_loader=test_dataloader,
                  lr_scheduler=scheduler )

In [24]:
trainer.early_stop = 30

train_config['Early stop'] = trainer.early_stop

In [ ]:
print(trainer.dir.split('/')[1])
print(f"{trainer.dir.split('/')[2]}_train")
print(model_dfl.__class__.__name__)

In [25]:
wandb.init( project=trainer.dir.split('/')[1], 
            name=f"{trainer.dir.split('/')[2]}_train", 
            config=train_config )

In [26]:
trainer.train()


Epoch : 0 | Train Loss : 0.21422 | Train P.A : 93.91% | Train IOU : 0.00000 | Val Loss : 0.07484 | Val P.A : 99.07% | Val IOU : 0.00000 | Training Time : 122.82sec


Train IOU,▁
Train Loss,▁
Train P.A,▁
Val IOU,▁
Val Loss,▁
Val P.A,▁
Train IOU,0.0
Train Loss,0.21422
Train P.A,93.90741
Val IOU,0.0
Val Loss,0.07484


Test Loss : 0.06999 | Test P.A : 99.21% | Test IOU : 0.00000 | 

Test IOU,▁
Test Loss,▁
Test P.A,▁
Test IOU,0.0
Test Loss,0.06999
Test P.A,99.21344
